In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [2]:
from util.proj1_helpers import *
from util.helpers import *
DATA_TRAIN_PATH ="data/train.csv" # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

## Do your thing crazy machine learning thing here :) ...

In [3]:
from machine_learning_tools.costs import *
from machine_learning_tools.least_squares import *
from machine_learning_tools.ridge_regression import *

In [4]:
fi=build_poly(tX,30)
weight = least_squares(y,fi)
print(weight)
rmse_ls = np.sqrt(2*compute_loss(y,fi,weight))
print(rmse_ls)

[ -3.63971757e+000   3.29487038e-002  -3.87659758e-004   1.35069058e-006
  -2.94739107e-010  -5.10729818e-012   7.05307748e-015  -4.31074928e-019
  -5.28203967e-021   4.10369887e-024  -1.09769624e-027   1.13682313e-032
   1.27302332e-035   1.42919875e-038  -2.76360999e-042  -3.86949235e-046
   1.47230382e-051   2.56111109e-053  -2.46135534e-057   8.70415804e-061
  -1.13996451e-064   3.56022035e-068  -2.15323412e-071   2.54828892e-075
  -3.95544994e-079   6.36794723e-083   8.97497561e-087   4.34936117e-090
  -9.36558854e-094  -1.81519857e-097   2.97132312e-101]
397.50254651


In [5]:
fi_r = build_poly(tX, 30) #degree should be much bigger than 30, but it takes long to execute
weight_r = ridge_regression(y, fi_r, 1000)
print(weight_r) #because values of fi function are very big, lambda factor doesn't do much and weight doesn't become sparse
                #as it should be: HINT: change function fi in a right way, or put really big lambda (10^100)
rmse_r = np.sqrt(2*compute_mse_ridge(y,fi_r,weight_r, 1000))
print(rmse_r)

[ -1.69581759e+000  -2.99890215e-002  -3.81976145e-004   1.88479509e-006
  -1.30949217e-009  -4.51629137e-012   7.50408297e-015  -1.32930798e-018
  -4.72806531e-021   3.95467260e-024  -1.08772244e-027   1.22641089e-032
   1.37183347e-035   1.39997606e-038  -2.76072620e-042  -3.85198197e-046
   1.76567184e-051   2.53040633e-053  -2.31041938e-057   8.55673242e-061
  -1.09387425e-064   3.30604418e-068  -2.11660022e-071   2.49285108e-075
  -3.87879781e-079   6.45444975e-083   9.17386464e-087   4.30360589e-090
  -9.55874934e-094  -1.78182602e-097   2.96299918e-101]
267.492920305


In [ ]:
#TODO : 
# 1) look at data to see if there are special values to be changed
#(for example there are a lot of -999 values in the train.csv i dont know what we should do with them)
# if nb of -999 is greater than x% drop the feature. otherwise learn about the distribution of the values to randomly
# assign values according to this distribution
# 2) standardize features (subtract mean like in lab1)
# 3) test first version to see if it works : use 'logistic_regression' on data and submit a first time on kaggle
# when it works, improve with following : 
# 4) do cross-validation on data
# 5) feature-processing : maybe some features are not necessary, or add features that are a power of an existing feature


## Generate predictions and save ouput in csv format for submission:

In [6]:
DATA_TEST_PATH = '' # TODO: download test data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

OSError:  not found.

In [ ]:
OUTPUT_PATH = '' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)